# Installing Required Libraries & Importing Modules

In [1]:
!pip install pytesseract pdf2image pillow json5 pandas tqdm psycopg2-binary python-dotenv transformers huggingface_hub torch accelerate datasets pymongo pymupdf pdfplumber PyPDF2 pdfminer.six

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/61/69/3b3d7bd583c6d3cbe5100802efa5beacaacc86e37b653fc708bf3d6853b8/psycopg2_binary-2.9.10-cp311-cp311-win_amd64.whl.metadata
  Using cached psycopg2_binary-2.9.10-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/5f/a0/d9ef19f780f319c21ee90ecfef4431cbeeca95bec7f14071785c17b6029b/accelerate-1.10.1-py3-none-any.whl.metadata
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/eb/62/eb8157afb21bd229c864521c1ab4fa8e9b4f1b06bafdd8c4668a7a31b5dd/datasets-4.0.0-py3-none-any.whl.metadata
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for dill<0.3.9,>=0.3.0 from https://files.pythonhosted.org/packages/c9/7a/cef76fd8438a42f96db64ddaa85280485a9c395e7df3db

ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\LENOVO\\Desktop\\ENSA\\GI3\\PFE\\resume-recommender\\ai\\.venv\\Lib\\site-packages\\psycopg2\\_psycopg.cp311-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Windows-compatible poppler installation
import platform
import os
import urllib.request
import zipfile
from pathlib import Path

if platform.system() == 'Windows':
    print("Setting up poppler for Windows...")
    
    # Check if poppler is already available
    poppler_path = r"C:\Program Files\poppler\bin"
    if not os.path.exists(poppler_path):
        print("Poppler not found in standard location. Checking environment...")
        # For Windows, we'll assume poppler-utils is installed or use pdf2image without it
        print("Note: If PDF processing fails, please install poppler-utils for Windows")
        print("Download from: https://github.com/oschwartz10612/poppler-windows/releases/")
    else:
        print(f"Poppler found at: {poppler_path}")
        # Add to PATH if not already there
        if poppler_path not in os.environ.get('PATH', ''):
            os.environ['PATH'] = poppler_path + ';' + os.environ.get('PATH', '')
            
print("Poppler setup complete")

Setting up poppler for Windows...
Poppler not found in standard location. Checking environment...
Note: If PDF processing fails, please install poppler-utils for Windows
Download from: https://github.com/oschwartz10612/poppler-windows/releases/
Poppler setup complete


In [3]:
!pip install -U bitsandbytes

^C


In [4]:
import os
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import re
import json5
import pandas as pd
from tqdm import tqdm
import psycopg2
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from huggingface_hub import login
from pymongo import MongoClient
import json
import torch
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import logging

# Loading Environment Variables

In [5]:
load_dotenv()

# Hugging Face Token (optional)
hf_token = os.getenv('HF_TOKEN')
if hf_token:
    print("Hugging Face token loaded")
else:
    print("No Hugging Face token found - some features may be limited")

# MongoDB Configuration (optional)
mongo_host = os.getenv('MONGO_HOST')
mongo_port = os.getenv('MONGO_PORT')
mongo_db = os.getenv('MONGO_DB')
mongo_collection = os.getenv('MONGO_COLLECTION')

# PostgreSQL Configuration (with defaults)
postgres_host = os.getenv('POSTGRES_HOST', 'localhost')
postgres_port = os.getenv('POSTGRES_PORT', '5432')
postgres_db = os.getenv('POSTGRES_DB', 'pfe_db')
postgres_user = os.getenv('POSTGRES_USER', 'postgres')
postgres_password = os.getenv('POSTGRES_PASSWORD', '1986')

print(f"PostgreSQL config: {postgres_host}:{postgres_port}/{postgres_db}")
print(f"MongoDB config: {mongo_host}:{mongo_port}/{mongo_db} (collection: {mongo_collection})")

No Hugging Face token found - some features may be limited
PostgreSQL config: localhost:5432/pfe_db
MongoDB config: localhost:27017/PFE (collection: resumes)


# Hugging Face Connection

In [6]:
# Hugging Face Login (optional)
if hf_token:
    try:
        login(hf_token)
        print("Hugging Face login successful")
    except Exception as e:
        print(f"Hugging Face login failed: {e}")
        print("Continuing without HF authentication - some features may be limited")
else:
    print("Skipping Hugging Face login - no token provided")

Skipping Hugging Face login - no token provided


# MongoDB Connection

In [7]:
# MongoDB Connection with error handling
client = None
db = None
resumes = None

try:
    # Check if MongoDB variables are properly set
    if mongo_host and mongo_port and mongo_db and mongo_collection:
        print(f"Connecting to MongoDB at {mongo_host}:{mongo_port}")
        client = MongoClient(f"{mongo_host}:{mongo_port}")
        db = client[mongo_db]
        resumes = db[mongo_collection]
        print("MongoDB connection successful")
    else:
        print("MongoDB configuration incomplete. Skipping MongoDB connection.")
        print(f"MONGO_HOST: {mongo_host}, MONGO_PORT: {mongo_port}")
        print(f"MONGO_DB: {mongo_db}, MONGO_COLLECTION: {mongo_collection}")
except Exception as e:
    print(f"MongoDB connection failed: {e}")
    print("Continuing without MongoDB. Results will only be saved to PostgreSQL.")
    client = None
    db = None
    resumes = None

Connecting to MongoDB at localhost:27017
MongoDB connection successful


# Postgres Connection

In [8]:
def get_postgres_connection():
    try:
        conn = psycopg2.connect(
            dbname=postgres_db,
            user=postgres_user,
            password=postgres_password,
            host=postgres_host,
            port=postgres_port
        )
        print("✅ Connected to PostgreSQL database.")
        return conn
    except Exception as e:
        print(f"❌ Error connecting to PostgreSQL: {e}")
        return None

# Loading Model and Tokenizer

In [9]:
# Smart Model Loading with Lightweight Fallback
import psutil
import torch

def check_system_resources():
    """Check if system has enough resources for heavy models"""
    # Get available RAM in GB
    available_ram = psutil.virtual_memory().available / (1024**3)
    
    # Check if CUDA is available
    has_gpu = torch.cuda.is_available()
    if has_gpu:
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    else:
        gpu_memory = 0
    
    print(f"💾 Available RAM: {available_ram:.1f} GB")
    print(f"🖥️  GPU Available: {has_gpu}")
    if has_gpu:
        print(f"🎮 GPU Memory: {gpu_memory:.1f} GB")
    
    # Determine if we can use heavy models
    can_use_heavy_model = (available_ram > 8 and gpu_memory > 6) or available_ram > 16
    return can_use_heavy_model, available_ram, has_gpu, gpu_memory

# Initialize model variables
model = None
tokenizer = None
chat_pipeline = None
use_light_extraction = False

# Check system resources
can_use_heavy, ram, has_gpu, gpu_mem = check_system_resources()

if can_use_heavy:
    print("🚀 Attempting to load heavy extraction model (Mixtral-8x7B)...")
    try:
        # Check for bitsandbytes
        import bitsandbytes
        
        drive_model_path = "/content/drive/MyDrive/PFE/ai_models/mixtral-8x7b"
        
        if os.path.exists(drive_model_path):
            print("Loading model from Drive cache...")
            tokenizer = AutoTokenizer.from_pretrained(drive_model_path)
            model = AutoModelForCausalLM.from_pretrained(
                drive_model_path,
                device_map="auto"
            )
        else:
            print("Downloading model from HuggingFace...")
            MODEL_NAME = "mistralai/Mixtral-8x7B-Instruct-v0.1"

            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.float16
            )

            tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
            model = AutoModelForCausalLM.from_pretrained(
                MODEL_NAME,
                quantization_config=bnb_config,
                device_map="auto"
            )

        # Set padding token if needed
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        # Create pipeline
        chat_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            device_map="auto",
            pad_token_id=tokenizer.pad_token_id,
            return_full_text=False
        )
        
        print("✅ Heavy extraction model loaded successfully!")
        
    except Exception as e:
        print(f"❌ Failed to load heavy model: {e}")
        print("🔄 Falling back to lightweight extraction...")
        use_light_extraction = True
        
else:
    print("⚡ System resources limited - using lightweight extraction")
    use_light_extraction = True

# Initialize lightweight extraction model if needed
light_model = None
light_tokenizer = None
light_pipeline = None

if use_light_extraction:
    print("🪶 Loading lightweight extraction model (DistilBERT-based)...")
    try:
        from transformers import pipeline as hf_pipeline
        
        # Use a much smaller, efficient model for text generation/extraction
        # Option 1: DistilGPT-2 (much smaller, ~82MB)
        light_model_name = "distilgpt2"
        
        # Alternative options if DistilGPT-2 doesn't work well:
        # "microsoft/DialoGPT-small" (~117MB)
        # "gpt2" (~500MB, still much smaller than Mixtral)
        
        light_pipeline = hf_pipeline(
            "text-generation",
            model=light_model_name,
            tokenizer=light_model_name,
            max_length=512,
            do_sample=True,
            temperature=0.7,
            return_full_text=False
        )
        
        print(f"✅ Lightweight model ({light_model_name}) loaded successfully!")
        print("📝 Note: Using rule-based extraction with AI assistance for better results")
        
    except Exception as e:
        print(f"⚠️ Even lightweight model failed: {e}")
        print("🛠️ Using pure rule-based extraction (no AI model)")
        light_pipeline = None

# Summary of what we're using
if chat_pipeline:
    print("\n🎯 Using: Heavy AI model (Mixtral-8x7B) for extraction")
elif light_pipeline:
    print(f"\n🎯 Using: Lightweight AI model ({light_model_name}) + rule-based extraction")
else:
    print("\n🎯 Using: Pure rule-based extraction (no AI models)")

💾 Available RAM: 1.0 GB
🖥️  GPU Available: False
⚡ System resources limited - using lightweight extraction
🪶 Loading lightweight extraction model (DistilBERT-based)...


Device set to use cpu


✅ Lightweight model (distilgpt2) loaded successfully!
📝 Note: Using rule-based extraction with AI assistance for better results

🎯 Using: Lightweight AI model (distilgpt2) + rule-based extraction


# Text extraction (OCR or PyMuPDF)

In [10]:
def extract_text(pdf_path):
    try:
        # Try PDFPlumber first (more accurate than PyMuPDF for many layouts)
        full_text = ""
        logging.getLogger("pdfminer").setLevel(logging.ERROR)
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    full_text += text + "\n"

        full_text = full_text.strip()

        # If text is empty or too short, fallback to OCR
        if not full_text or len(full_text) < 50:
            print("🔁 Switching to OCR (image-based or low-quality PDF)")
            images = convert_from_path(pdf_path)
            full_text = ""
            for i, img in enumerate(images):
                ocr_result = pytesseract.image_to_string(img)
                full_text += ocr_result + "\n"

        return full_text.strip()

    except Exception as e:
        print(f"❌ Error during text extraction: {e}")
        return None

# Clean up the text

In [11]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Prompt creation


In [12]:
def create_prompt(resume_text):
    prompt = f"""
Extract structured data from the following resume text and return a valid JSON object with the fields:

- name, age, email, phone, personal_links (github, linkedin, portfolio, indeed, others), skills, languages, experiences (job_title, company, duration, description), projects (project_title, project_summary), education (degree, institution, year), certifications, years_of_experience.

Return only a **raw JSON object**. Do NOT include any explanation, markdown, or code block (like ```json or ```python). Just the JSON like this example (no comments or extra text):
Example:
{{
  "name": "Jane Doe",
  "age": 28,
  "email": "jane.doe@example.com",
  "phone": "+212612345678",
  "personal_links": {{
    "github": "https://github.com/janedoe",
    "linkedin": "https://linkedin.com/in/janedoe",
    "portfolio": "https://janedoe.dev",
    "indeed": "https://indeed.com/profile/janedoe",
    "others": ["https://behance.net/janedoe", "https://medium.com/@janedoe"]
  }},
  "skills": ["Python", "React", "MongoDB", "Docker", "problem solving", "communication"],
  "languages": ["English", "Arabic", "French"],
  "experiences": [
    {{
      "job_title": "AI Engineer",
      "company": "TechCorp",
      "duration": "Jan 2020 - Present",
      "description": "Led a team of engineers to develop NLP models for document parsing."
    }}
  ],
  "projects": [
    {{
      "project_title": "Resume Parser",
      "project_summary": "Developed a parser using Transformers and MongoDB."
    }}
  ],
  "education": [
    {{
      "degree": "BSc in Computer Engineering",
      "institution": "ENSA",
      "year": "2022"
    }}
  ],
  "certifications": ["AWS Certified ML Engineer"],
  "years_of_experience": 5
}}

Resume Text:
{resume_text}
"""
    return prompt

# Call Hugging Face model

In [13]:
# Smart AI Model Calling with Fallbacks
def call_AI_model(prompt):
    """Call the appropriate AI model based on what's available"""
    
    # Try heavy model first
    if chat_pipeline:
        try:
            output = chat_pipeline(
                prompt,
                max_new_tokens=1200,
                do_sample=True,
                temperature=0.2,
                return_full_text=False
            )[0]['generated_text']
            return output
        except Exception as e:
            print(f"❌ Heavy model failed: {e}")
            print("🔄 Falling back to lightweight extraction...")
    
    # For lightweight models, we use a hybrid approach
    if light_pipeline:
        try:
            print("🪶 Using lightweight model + rule-based extraction...")
            
            # Use rule-based as primary, AI as enhancement
            rule_based_result = extract_resume_data_rule_based(prompt.split("Resume text:")[-1])
            
            # Try to enhance with lightweight AI for specific fields
            try:
                enhance_prompt = f"Extract skills from: {prompt[-300:]}\nSkills:"
                ai_enhancement = light_pipeline(
                    enhance_prompt,
                    max_length=100,
                    do_sample=True,
                    temperature=0.3,
                    return_full_text=False
                )[0]['generated_text']
                
                print(f"🤖 AI Enhancement: {ai_enhancement}")
            except Exception as enhance_error:
                print(f"⚠️ AI enhancement failed: {enhance_error}")
            
            return rule_based_result
            
        except Exception as e:
            print(f"❌ Lightweight model failed: {e}")
            print("🛠️ Falling back to pure rule-based extraction...")
    
    # Fallback to rule-based extraction
    print("🔧 Using pure rule-based extraction (no AI model)")
    resume_text = prompt.split("Resume text:")[-1] if "Resume text:" in prompt else prompt
    return extract_resume_data_rule_based(resume_text)

In [14]:
def extract_resume_data_rule_based(resume_text):
    """Rule-based resume data extraction using regex patterns"""
    import re
    import json
    
    # Initialize result structure
    result = {
        "name": "",
        "email": "",
        "phone": "",
        "education": [],
        "work_experience": [],
        "skills": [],
        "languages": [],
        "certifications": []
    }
    
    # Extract email
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    email_match = re.search(email_pattern, resume_text, re.IGNORECASE)
    if email_match:
        result["email"] = email_match.group(0)
    
    # Extract phone number
    phone_patterns = [
        r'\+?\d{1,4}[-.\s]?\(?\d{1,3}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,9}',
        r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b',
        r'\(\d{3}\)\s*\d{3}[-.]?\d{4}'
    ]
    
    for pattern in phone_patterns:
        phone_match = re.search(pattern, resume_text)
        if phone_match:
            result["phone"] = phone_match.group(0).strip()
            break
    
    # Extract name (usually at the beginning)
    lines = resume_text.split('\n')
    for line in lines[:5]:  # Check first 5 lines
        line = line.strip()
        if len(line) > 2 and len(line) < 50 and not re.search(r'[@\d\+]', line):
            # Remove common prefixes
            clean_line = re.sub(r'^(resume|cv|curriculum vitae|profile)\s*:?\s*', '', line, flags=re.IGNORECASE)
            if clean_line and len(clean_line.split()) >= 2:
                result["name"] = clean_line
                break
    
    # Extract skills (look for skills section)
    skills_pattern = r'(?:skills?|technical skills?|core competencies|technologies)[\s\S]*?(?=\n\s*[A-Z][a-z]+|\Z)'
    skills_match = re.search(skills_pattern, resume_text, re.IGNORECASE | re.MULTILINE)
    if skills_match:
        skills_text = skills_match.group(0)
        # Extract individual skills
        skill_items = re.findall(r'\b[A-Za-z][A-Za-z\s\-\+#\.]*[A-Za-z]\b', skills_text)
        result["skills"] = [skill.strip() for skill in skill_items if len(skill.strip()) > 2][:10]
    
    # Extract education
    edu_pattern = r'(?:education|academic|qualification)[\s\S]*?(?=\n\s*[A-Z][a-z]+|\Z)'
    edu_match = re.search(edu_pattern, resume_text, re.IGNORECASE | re.MULTILINE)
    if edu_match:
        edu_text = edu_match.group(0)
        # Look for degree patterns
        degree_patterns = [
            r'(?:bachelor|master|phd|doctorate|diploma|certificate).*?(?:in|of).*?(?:\d{4}|\n)',
            r'(?:b\.?s\.?|m\.?s\.?|b\.?a\.?|m\.?a\.?|phd).*?(?:\d{4}|\n)'
        ]
        
        for pattern in degree_patterns:
            degrees = re.findall(pattern, edu_text, re.IGNORECASE)
            for degree in degrees:
                result["education"].append({
                    "degree": degree.strip(),
                    "institution": "",
                    "year": ""
                })
    
    # Extract work experience
    exp_pattern = r'(?:experience|employment|work history|professional experience)[\s\S]*?(?=\n\s*[A-Z][a-z]+|\Z)'
    exp_match = re.search(exp_pattern, resume_text, re.IGNORECASE | re.MULTILINE)
    if exp_match:
        exp_text = exp_match.group(0)
        # Simple job extraction
        job_lines = [line.strip() for line in exp_text.split('\n') if line.strip()]
        for line in job_lines[:5]:  # Limit to 5 entries
            if len(line) > 10 and not line.lower().startswith(('experience', 'work', 'employment')):
                result["work_experience"].append({
                    "title": line,
                    "company": "",
                    "duration": "",
                    "description": ""
                })
    
    # Convert to JSON string similar to AI model output
    return json.dumps(result, indent=2)

In [15]:
# Test the intelligent model system
sample_resume = """
John Smith
Software Engineer
john.smith@email.com
+1-555-123-4567

EXPERIENCE
Senior Software Developer at Tech Corp (2020-2023)
- Developed web applications using React and Node.js
- Led a team of 5 developers
- Implemented CI/CD pipelines

Junior Developer at StartupXYZ (2018-2020)
- Built mobile apps using React Native
- Worked with Python and Django

EDUCATION
Master of Science in Computer Science
Stanford University (2018)

Bachelor of Science in Computer Science  
MIT (2016)

SKILLS
Python, JavaScript, React, Node.js, SQL, MongoDB, AWS, Docker, Git

LANGUAGES
English (Native)
Spanish (Fluent)
French (Intermediate)
"""

print("🧪 Testing intelligent model system...")
print("=" * 60)

# Test the model calling function
test_prompt = f"""Extract key information from this resume and return it as JSON with the following structure:
{{
    "name": "Full name",
    "email": "Email address",
    "phone": "Phone number",
    "skills": ["skill1", "skill2"],
    "education": [
        {{
            "degree": "Degree name",
            "institution": "University name", 
            "year": "Year"
        }}
    ],
    "work_experience": [
        {{
            "title": "Job title",
            "company": "Company name",
            "duration": "Years",
            "description": "Job description"
        }}
    ],
    "languages": ["language1", "language2"],
    "certifications": ["cert1", "cert2"]
}}

Resume text:
{sample_resume}"""

result = call_AI_model(test_prompt)
print("🎯 Extraction Result:")
print("=" * 30)
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🧪 Testing intelligent model system...
🪶 Using lightweight model + rule-based extraction...
⚠️ AI enhancement failed: Input length of input_ids is 100, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.
🎯 Extraction Result:
{
  "name": "John Smith",
  "email": "john.smith@email.com",
  "phone": "+1-555-123-4567",
  "education": [],
  "work_experience": [],
  "skills": [
    "SKILLS"
  ],
  "languages": [],
  "certifications": []
}


# Parse model response

In [16]:
def parse_AI_model_output(ai_output):
    try:
        json_str = ai_output.strip().strip('```json').strip('```').strip()
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        return None

# Save to MongoDB

In [17]:
def save_to_mongodb(resume):
    try:
        resumes.insert_one(resume)
        print("✅ Saved to MongoDB!")
    except Exception as e:
        print(f"❌ Error saving to MongoDB: {e}")

# Full resume processing

In [18]:
def process_resume(pdf_path, uuid=None):
    resume_text = extract_text(pdf_path)
    if not resume_text:
        print("❌ No text found in resume. Skipping.")
        return False

    resume_text = preprocess_text(resume_text)
    print("📄 Extracted and preprocessed text:", resume_text)
    prompt = create_prompt(resume_text)
    AI_model_output = call_AI_model(prompt)
    print("ai model output:", AI_model_output)
    parsed_data = parse_AI_model_output(AI_model_output)
    print("parsed data output:", parsed_data)

    if parsed_data:
        parsed_data['uuid'] = uuid
        save_to_mongodb(parsed_data)
        print(f"✅ Successfully processed and stored resume UUID {uuid}")
        return True
    else:
        print("❌ Failed to parse resume.")
        return False

# Process all resumes from Postgres database

In [19]:
def update_parsed_flag(resume_id):
    try:
        conn = get_postgres_connection()
        cursor = conn.cursor()
        cursor.execute("UPDATE resumes SET parsed = TRUE WHERE id = %s;", (resume_id,))
        conn.commit()
        conn.close()
        return True
    except Exception as e:
        print(f"❌ Failed to update resume ID {resume_id}: {e}")
        return False

In [20]:
def process_all_resumes_from_db():
    conn = get_postgres_connection()
    cursor = conn.cursor()

    cursor.execute("SELECT resume_id, uuid, storage_path FROM resumes WHERE parsed = FALSE;")

    failed_resumes = []
    rows = cursor.fetchall()

    if not rows:
        print("✅ No new resumes to process.")
        return

    for row in tqdm(rows, desc="Processing Resumes"):
        resume_id, uuid, file_path = row
        print(f"\n🔍 Processing resume ID {resume_id}: {file_path}")
        try:
            success = process_resume(file_path, uuid)
            if success:
                update_parsed_flag(resume_id)
        except Exception as e:
            print(f"❌ Unexpected error while processing {file_path}: {e}")
            failed_resumes.append(file_path)

    conn.close()

    if failed_resumes:
        print("\n⚠️ Failed to process the following resumes:")
        for path in failed_resumes:
            print(f" - {path}")

# Run the processing function

In [21]:
process_all_resumes_from_db()

✅ Connected to PostgreSQL database.


Processing Resumes:   0%|                                                   | 0/43 [00:00<?, ?it/s]


🔍 Processing resume ID 7: resumes/07df5960-c4bc-4f2f-8e2c-010f10fffdf1_15.pdf


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


📄 Extracted and preprocessed text: Kyoti Khan Address: House No-209, Email: kyotikhan@gmail.com Sector 13 A, Gurugram Mobile: 09028202799, 09299367700 Haryana, 122000 RESEARCH AND TECHNICAL SKILLS Research Interests Soft Computing, Image Processing, Medical image analysis, Marketing, Data Science, Analytics, Computer Vision, Neural Networks, Speech processing, Deep learning, Fuzzy logic, Machine learning, Artificial intelligence, Natural Language Processing (word2vec, RNNs, transformer neural networks, BERT, GPT), Computer-aided diagnosis, Ultrasound imaging etc. Engineering Competences Computer Science: problem analysis, modelling of the solution by use of algorithms, Maths, Probability, statistics and algorithms, Neural Networks, transfer learning, convolutional neural networks, deep learning architectures, type II fuzzy set, Intuitionistic fuzzy sets, fuzzy classifiers, machine learning- based classifiers. o Operating Systems: Windows 10/ 7/xp. o Simulation Software: Matlab/ Simulin

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


📄 Extracted and preprocessed text: MOHAMED AMIN EL METNI Computer Engineer Education (2022 - present) ENSA-TETOUAN 3rd year of a computer engineering program. (2020 - 2022) CPGE - TANGER Two years of preparatory classes in Physics, Chemistry and Engineering Sciences (PCSI - PSI). About me (2019 - 2020) HIGH SCHOOL KAADI AYAD - TÉTOUAN I'm a 23 years old computer engineer Baccalaureate in Mathematical Sciences (Option B), with honors with strong analytical skills and creative design thinking. Team player Professional Experience with an eye for details. (March 2025 - August 2025) Contact AI ENGINEER & FULL STACK DEVELOPER NTT Data - Morocco, Tetouan amin-elmetni HireLens – AI-Powered Resume Recommendation System. Designed and implemented a core resume scoring engine amin.elmetni@gmail.com using LLMs, NLP, and fine-tuning techniques to accurately +212 694 39 90 71 evaluate and classify resumes by user-defined criteria. Jamaa Mezouak, Tetouan Built a secure and intuitive web interface for 

Processing Resumes: 100%|██████████████████████████████████████████| 43/43 [02:32<00:00,  3.18s/it]

Processing Resumes: 100%|██████████████████████████████████████████| 43/43 [02:32<00:00,  3.56s/it]

🤖 AI Enhancement: 




























ai model output: {
  "name": "",
  "email": "jane.doe@example.com",
  "phone": "+212612345678",
  "education": [],
  "work_experience": [],
  "skills": [
    "skills",
    "languages",
    "experiences",
    "company",
    "duration",
    "description",
    "projects",
    "education",
    "degree",
    "institution"
  ],
  "languages": [],
  "certifications": []
}
parsed data output: {'name': '', 'email': 'jane.doe@example.com', 'phone': '+212612345678', 'education': [], 'work_experience': [], 'skills': ['skills', 'languages', 'experiences', 'company', 'duration', 'description', 'projects', 'education', 'degree', 'institution'], 'languages': [], 'certifications': []}
✅ Saved to MongoDB!
✅ Successfully processed and stored resume UUID e6f2dd76-6f5f-4a43-b5fc-f9812b8e5ae5
✅ Connected to PostgreSQL database.
❌ Failed to update resume ID 44: ERREUR:  la colonne « id » n'existe pas
LINE 1: UPDATE resumes SET parsed = TRUE WHERE id = 44;
      